# Wetlands LLM + MCP Testing

Test the LLM with MCP server integration using LangChain

In [15]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.tools import tool

## Configuration

Using remote MCP server at nrp-nautilus.io

In [16]:
# Load config
with open('../maplibre/config.json') as f:
    config = json.load(f)

# Get API key from environment
api_key = os.getenv('NRP_API_KEY')
if not api_key:
    print("WARNING: NRP_API_KEY not set!")
    api_key = input("Enter your NRP API key: ")

# Remote MCP server URL (hosted on nrp-nautilus.io)
mcp_url = "https://biodiversity-mcp.nrp-nautilus.io/sse"
llm_endpoint = config['llm_host']  # Use llm_host for OpenAI-compatible endpoint
llm_model = "glm-v" # config['llm_model']

print(f"LLM Endpoint: {llm_endpoint}")
print(f"Model: {llm_model}")
print(f"MCP Server: {mcp_url}")

LLM Endpoint: https://ellm.nrp-nautilus.io/v1
Model: glm-v
MCP Server: https://biodiversity-mcp.nrp-nautilus.io/sse


## Load System Prompt

In [17]:
with open('system-prompt.md') as f:
    system_prompt = f.read()

print(f"System prompt loaded: {len(system_prompt)} characters")

System prompt loaded: 5776 characters


## Setup MCP Integration with SSE Transport

Connect to the remote MCP server using SSE (Server-Sent Events) transport:

In [18]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# Connect to the remote MCP server's SSE endpoint
client = MultiServerMCPClient(
    {
        "wetlands": {
            "transport": "sse",
            "url": mcp_url,
        }
    }
)

# Get tools from the MCP server
tools = await client.get_tools()

print(f"✓ Connected to remote MCP server!")
print(f"✓ Available tools: {[tool.name for tool in tools]}")

✓ Connected to remote MCP server!
✓ Available tools: ['query']


## Initialize LLM with MCP Tools

In [19]:
# Initialize LangChain LLM with MCP tools
llm = ChatOpenAI(
    base_url=llm_endpoint,
    api_key=api_key,
    model=llm_model,
    temperature=0.7
)

# Bind the MCP tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print(f"LLM initialized with {len(tools)} MCP tools")

LLM initialized with 1 MCP tools


## Test LLM with System Prompt (No Tools)

Test that the LLM responds correctly with the system prompt but without using any tools:

In [20]:
# Test LLM without tools, just with system prompt
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="What are some types of wetland?")
]

response = llm.invoke(messages)
print("LLM Response (without tools):")
print(response.content)

LLM Response (without tools):

# Types of Wetlands

Based on the global wetlands dataset, wetlands are categorized into several main types:

## 1. Open Water (codes 1-5)
- Freshwater lakes
- Saline lakes
- Reservoirs
- Large rivers
- Small rivers

## 2. Lacustrine Wetlands (codes 19-20)
- Lacustrine fringe (wetlands around lake edges)
- Lacustrine marsh (marshes associated with lakes)

## 3. Riverine Wetlands (codes 16-18)
- Floodplain (areas that flood during high water)
- Oxbow lakes (curved lakes formed from river meanders)
- Riverine wetland (wetlands along river channels)

## 4. Palustrine Wetlands (codes 13-15)
- Freshwater marsh (marshes with predominantly freshwater)
- Swamp forest (forested wetlands)
- Flooded forest (forests that are regularly inundated)

## 5. Ephemeral Wetlands (codes 21-23)
- Pan (shallow depressions that hold water seasonally)
- Intermittent wetland (wetlands that dry out periodically)
- Seasonal wetland (wetlands that appear only during wet seasons)

## 

## Example: Direct Tool Test

Test an MCP tool directly (without the LLM generating the query):

In [21]:
# Find the query tool
query_tool = next((t for t in tools if t.name == "query"), None)

if query_tool:
    # Test it with a simple query
    test_query = """
    SET THREADS=100;
    INSTALL httpfs; LOAD httpfs;
    CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path');
    SELECT COUNT(*) as total_hexagons 
    FROM read_parquet('s3://public-wetlands/hex/**')
    WHERE Z > 0
    LIMIT 10;
    """
    
    # Re-create client connection before invoking
    # This ensures we have a fresh session
    fresh_client = MultiServerMCPClient(
        {
            "wetlands": {
                "transport": "sse",
                "url": mcp_url,
            }
        }
    )
    fresh_tools = await fresh_client.get_tools()
    fresh_query_tool = next((t for t in fresh_tools if t.name == "query"), None)
    
    result = await fresh_query_tool.ainvoke({"query": test_query})
    print("Result:")
    print(result)
else:
    print("Query tool not found!")


Result:
+----------------+
| total_hexagons |
|     BIGINT     |
+----------------+
|   647463329    |
+----------------+


## Example: LLM with MCP Tool Calling

Now let's use the LLM to automatically generate SQL queries and call the MCP tools:

In [22]:
async def ask_wetlands_question(question: str):
    """Ask a natural language question about wetlands data"""
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=question)
    ]
    
    response = await llm_with_tools.ainvoke(messages)
    
    print(f"\n{'='*70}")
    print(f"❓ {question}")
    print(f"{'='*70}\n")
    
    # Debug: Show what the LLM returned
    print(f"🔍 DEBUG: response.tool_calls = {response.tool_calls}")
    print(f"🔍 DEBUG: response.content = {response.content[:200] if response.content else 'None'}...")
    print()
    
    # Handle tool calls
    if response.tool_calls:
        for tool_call in response.tool_calls:
            print(f"🔧 Generating and executing SQL query...\n")
            print(tool_call['args']['query'])
            print("-" * 70)
            
            # Execute the tool with a fresh client connection
            try:
                # Create a fresh client for each tool invocation to avoid session timeouts
                fresh_client = MultiServerMCPClient(
                    {
                        "wetlands": {
                            "transport": "sse",
                            "url": mcp_url,
                        }
                    }
                )
                fresh_tools = await fresh_client.get_tools()
                fresh_tool = next((t for t in fresh_tools if t.name == tool_call['name']), None)
                
                if fresh_tool:
                    result = await fresh_tool.ainvoke(tool_call['args'])
                    
                    print(f"\n📊 Raw result:")
                    print(result)
                    print("-" * 70)
                    
                    # Get LLM interpretation
                    messages.append(response)
                    messages.append(HumanMessage(content=f"Tool result: {result}"))
                    final = await llm.ainvoke(messages)
                    
                    print(f"\n💬 Answer:")
                    print(final.content)
                    return final.content
                else:
                    print(f"❌ Tool '{tool_call['name']}' not found")
                    return None
            except Exception as e:
                print(f"❌ Error: {e}")
                import traceback
                traceback.print_exc()
                return None
    else:
        print(f"💬 {response.content}")
        return response.content


## Example Questions

Ask natural language questions - the LLM will generate SQL and use the MCP tools automatically:

In [23]:
await ask_wetlands_question("How many hectares of wetlands are there in total?")


❓ How many hectares of wetlands are there in total?

🔍 DEBUG: response.tool_calls = [{'name': 'query', 'args': {'query': "SET THREADS=100; INSTALL httpfs; LOAD httpfs; CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path'); SELECT COUNT(*) * 73.7327598 AS total_hectares FROM read_parquet('s3://public-wetlands/hex/**') WHERE Z > 0;"}, 'id': 'chatcmpl-tool-fad1a61c72124883b0fb7bac078b85f4', 'type': 'tool_call'}]
🔍 DEBUG: response.content = 
I'll calculate the total wetland area by counting all wetland hexagons and converting to hectares using the H3 resolution 8 area constant.
...

🔧 Generating and executing SQL query...

SET THREADS=100; INSTALL httpfs; LOAD httpfs; CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path'); SELECT COUNT(*) * 73.7327598 AS total_hectares FROM read_parquet('s3://public-wetlands/hex/**') WHERE Z > 0;
----------------------------------------------------------------------

📊 Raw result

"\nBased on the global wetlands data, there are approximately **47.74 billion hectares** of wetlands worldwide.\n\nThis calculation includes all wetland types (open water, peatlands, palustrine, riverine, lacustrine, ephemeral, and coastal wetlands) across the entire globe. The figure represents the total area covered by wetlands as identified in the global dataset.\n\nTo put this in perspective:\n- This is roughly equivalent to **477 million square kilometers** (or about 184 million square miles)\n- It represents a significant portion of Earth's land surface, as wetlands are known to cover approximately 6% of the planet's land area\n\nThis comprehensive wetland area provides critical ecosystem services including water filtration, flood control, carbon sequestration, and habitat for diverse plant and animal species."

In [26]:
await ask_wetlands_question("Compute the fraction of the total area is each wetland code?")


❓ Compute the fraction of the total area is each wetland code?

🔍 DEBUG: response.tool_calls = [{'name': 'query', 'args': {}, 'id': 'chatcmpl-tool-e1a02b867d9541e3ba4e3aef3cc7b71a', 'type': 'tool_call'}]
🔍 DEBUG: response.content = 
I'll help you compute the fraction of total area for each wetland code. Let me create a query that calculates the area for each wetland type and then determines what fraction it represents of the tot...

🔧 Generating and executing SQL query...



KeyError: 'query'

In [25]:
# Example 2: Peatlands
await ask_wetlands_question("What is the total area of peatlands in square kilometers?")


❓ What is the total area of peatlands in square kilometers?

🔍 DEBUG: response.tool_calls = [{'name': 'query', 'args': {'query': "SET THREADS=100;\nINSTALL httpfs; LOAD httpfs;\nCREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path');\n\nSELECT \n    'Peatlands (codes 24-31)' as wetland_group,\n    COUNT(*) as total_hexagons,\n    ROUND(COUNT(*) * 73.7327598, 2) as total_hectares,\n    ROUND(COUNT(*) * 0.737327598, 2) as total_km2,\n    ROUND(COUNT(*) * 0.284679, 2) as total_sq_miles\nFROM read_parquet('s3://public-wetlands/hex/**')\nWHERE Z BETWEEN 24 AND 31;"}, 'id': 'chatcmpl-tool-e0440b82fcbe44f1823251da2c836dfb', 'type': 'tool_call'}]
🔍 DEBUG: response.content = 
I'll calculate the total area of peatlands globally using the wetlands dataset. Peatlands are represented by codes 24-31 in the Z column.
...

🔧 Generating and executing SQL query...

SET THREADS=100;
INSTALL httpfs; LOAD httpfs;
CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.c

"\nBased on the analysis of the global wetlands dataset, the total area of peatlands worldwide is **100,672,986 square kilometers** (or approximately 100.7 million km²).\n\nThis represents an enormous area of peatlands distributed across the globe, covering about 7% of Earth's land surface. The calculation is based on H3 hexagons (resolution 8) that fall within peatland categories (codes 24-31), which include bogs, fens, mires, and other peat-forming wetland types.\n\nTo put this in perspective:\n- This area is roughly equivalent to the size of Africa (about 30.4 million km²) multiplied by 3.3\n- It represents a significant carbon sink, as peatlands store vast amounts of carbon in their soils\n- These wetlands play crucial roles in water regulation, biodiversity support, and climate regulation\n\nWould you like to explore specific regions or types of peatlands in more detail?"

## Debug: Check for Duplicate Hexagons

Let's verify if there are duplicate h8 records in the data:

In [28]:
# Check sample of data to understand structure
query_tool = next((t for t in tools if t.name == "query"), None)

if query_tool:
    test_query = """
    SET THREADS=100;
    INSTALL httpfs; LOAD httpfs;
    CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path');
    
    SELECT 
        h8,
        Z,
        COUNT(*) as record_count
    FROM read_parquet('s3://public-wetlands/hex/**')
    WHERE Z BETWEEN 24 AND 31
    GROUP BY h8, Z
    HAVING COUNT(*) > 1
    ORDER BY record_count DESC
    LIMIT 10;
    """
    
    fresh_client = MultiServerMCPClient(
        {
            "wetlands": {
                "transport": "sse",
                "url": mcp_url,
            }
        }
    )
    fresh_tools = await fresh_client.get_tools()
    fresh_query_tool = next((t for t in fresh_tools if t.name == "query"), None)
    
    result = await fresh_query_tool.ainvoke({"query": test_query})
    print("Sample of duplicate h8 records:")
    print(result)
else:
    print("Query tool not found!")

Sample of duplicate h8 records:
+-----------------+---------+--------------+
|       h8        |    Z    | record_count |
|     VARCHAR     | INTEGER |    BIGINT    |
+-----------------+---------+--------------+
| 8800d2c8d7fffff |   31    |      54      |
| 8800d2cf2dfffff |   31    |      54      |
| 8800d2cabdfffff |   31    |      54      |
| 8800d2d9e3fffff |   31    |      54      |
| 880525b621fffff |   31    |      54      |
| 8805253689fffff |   31    |      54      |
| 8800d2d9b9fffff |   31    |      54      |
| 880525a4bdfffff |   31    |      52      |
| 8800d2da5bfffff |   31    |      52      |
| 8800d2dae1fffff |   31    |      52      |
+-----------------+---------+--------------+


## Reload System Prompt (After Fix)

The system prompt has been updated to use `COUNT(DISTINCT h8)` to handle duplicate records:

In [ ]:
# Reload the updated system prompt
with open('system-prompt.md') as f:
    system_prompt = f.read()

print(f"System prompt reloaded: {len(system_prompt)} characters")
print("\nKey change: Now using COUNT(DISTINCT h8) to handle duplicate records")

In [13]:
# Example 3: Top wetland types
await ask_wetlands_question("What are the top 5 most common wetland types by area?")


❓ What are the top 5 most common wetland types by area?

🔍 DEBUG: response.tool_calls = [{'name': 'query', 'args': {'query': "SET THREADS=100;\nINSTALL httpfs; LOAD httpfs;\nCREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path');\n\nSELECT \n    CASE \n        WHEN Z BETWEEN 1 AND 5 THEN 'Open Water'\n        WHEN Z BETWEEN 24 AND 31 THEN 'Peatlands'\n        WHEN Z BETWEEN 13 AND 15 THEN 'Palustrine'\n        WHEN Z BETWEEN 16 AND 18 THEN 'Riverine'\n        WHEN Z BETWEEN 19 AND 20 THEN 'Lacustrine'\n        WHEN Z BETWEEN 21 AND 23 THEN 'Ephemeral'\n        WHEN Z IN (6,7,8,9,10,11,12,32,33) THEN 'Coastal & Other'\n    END as wetland_type,\n    COUNT(*) as hex_count,\n    ROUND(COUNT(*) * 73.7327598, 2) as area_hectares,\n    ROUND(COUNT(*) * 0.737327598, 2) as area_km2\nFROM read_parquet('s3://public-wetlands/hex/**')\nWHERE Z > 0\nGROUP BY wetland_type\nORDER BY area_km2 DESC\nLIMIT 5;"}, 'id': 'chatcmpl-tool-b775d636f4074a4fa9494ee969d8eba9'

'\nBased on the global wetlands data, here are the top 5 most common wetland types by area:\n\n1. **Ephemeral wetlands** - 173,119,596.87 km² (173 billion hectares)\n   - These are temporary wetlands that form seasonally or after rainfall events\n\n2. **Peatlands** - 100,672,986.36 km² (100.7 billion hectares)\n   - Includes bogs, fens, and mires with significant carbon storage\n\n3. **Coastal & Other wetlands** - 89,257,468.87 km² (89.3 billion hectares)\n   - Encompasses coastal lagoons, estuaries, salt marshes, and other coastal ecosystems\n\n4. **Palustrine wetlands** - 54,224,595.61 km² (54.2 billion hectares)\n   - Freshwater marshes, swamps, and flooded forests\n\n5. **Open Water** - 45,906,019.94 km² (45.9 billion hectares)\n   - Includes freshwater lakes, saline lakes, reservoirs, and rivers\n\nThese results show that ephemeral wetlands dominate global wetland coverage, likely due to their widespread distribution in arid and semi-arid regions. Peatlands represent the second la

## Tips

- The LLM automatically generates SQL queries based on your questions
- The remote MCP server at nrp-nautilus.io handles database connections and S3 access
- Results are interpreted by the LLM in natural language
- Check `data-dictionary.md` for available fields and schema
- This tests the same MCP server that the chat.js frontend will use